[![AnalyticsDojo](https://s3.amazonaws.com/analyticsdojo/logo/final-logo.png)](http://rpi.analyticsdojo.com)
<center><h1>Bag of Words</h1></center>
<center><h3><a href = 'http://rpi.analyticsdojo.com'>rpi.analyticsdojo.com</a></h3></center>

This is adopted from: [Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-1-for-beginners-bag-of-words)
[https://github.com/wendykan/DeepLearningMovies](https://github.com/wendykan/DeepLearningMovies)


In [1]:
import nltk
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

In [2]:
train = pd.read_csv('../input/labeledTrainData.tsv', header=0, \
                    delimiter="\t", quoting=3)
unlabeled_train= pd.read_csv('../input/unlabeledTrainData.tsv', header=0, \
                    delimiter="\t", quoting=3)
test = pd.read_csv('../input/testData.tsv', header=0, \
                    delimiter="\t", quoting=3)

In [3]:
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np

In [4]:
print(train.columns.values, test.columns.values)

['id' 'sentiment' 'review'] ['id' 'review']


In [5]:
train.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [6]:
print('The train shape is: ', train.shape)
print('The train shape is: ', test.shape)

The train shape is:  (25000, 3)
The train shape is:  (25000, 2)


In [7]:
print('The first review is:')
print(train["review"][0])


The first review is:
"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bi

In [8]:
# Import BeautifulSoup into your workspace
from bs4 import BeautifulSoup             

# Initialize the BeautifulSoup object on a single movie review     
example1 = BeautifulSoup(train["review"][0], "html.parser" )  
print(example1.get_text())

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally starts is only on for 20 mi

In [9]:
import re
# Use regular expressions to do a find-and-replace
letters_only = re.sub("[^a-zA-Z]",           # The pattern to search for
                      " ",                   # The pattern to replace it with
                      example1.get_text() )  # The text to search
print (letters_only)

 With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him The actual feature film bit when it finally starts is only on for    mi

In [10]:
lower_case = letters_only.lower()        # Convert to lower case
words = lower_case.split()               # Split into words


In [11]:
# Enter Download then stopwords.
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
print (stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'no

In [13]:
# Remove stop words from "words"
words = [w for w in words if not w in stopwords.words("english")]
print (words)

['stuff', 'going', 'moment', 'mj', 'started', 'listening', 'music', 'watching', 'odd', 'documentary', 'watched', 'wiz', 'watched', 'moonwalker', 'maybe', 'want', 'get', 'certain', 'insight', 'guy', 'thought', 'really', 'cool', 'eighties', 'maybe', 'make', 'mind', 'whether', 'guilty', 'innocent', 'moonwalker', 'part', 'biography', 'part', 'feature', 'film', 'remember', 'going', 'see', 'cinema', 'originally', 'released', 'subtle', 'messages', 'mj', 'feeling', 'towards', 'press', 'also', 'obvious', 'message', 'drugs', 'bad', 'kay', 'visually', 'impressive', 'course', 'michael', 'jackson', 'unless', 'remotely', 'like', 'mj', 'anyway', 'going', 'hate', 'find', 'boring', 'may', 'call', 'mj', 'egotist', 'consenting', 'making', 'movie', 'mj', 'fans', 'would', 'say', 'made', 'fans', 'true', 'really', 'nice', 'actual', 'feature', 'film', 'bit', 'finally', 'starts', 'minutes', 'excluding', 'smooth', 'criminal', 'sequence', 'joe', 'pesci', 'convincing', 'psychopathic', 'powerful', 'drug', 'lord', 

In [14]:
#Now we are going to do our first class
class KaggleWord2VecUtility(object):
    """KaggleWord2VecUtility is a utility class for processing raw HTML text into segments for further learning"""

    @staticmethod
    def review_to_wordlist( review, remove_stopwords=False ):
        # Function to convert a document to a sequence of words,
        # optionally removing stop words.  Returns a list of words.
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review,"html.parser"  ).get_text()
        #
        # 2. Remove non-letters
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
        #
        # 3. Convert words to lower case and split them
        words = review_text.lower().split()
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]
        #
        # 5. Return a list of words
        return(words)

    # Define a function to split a review into parsed sentences
    @staticmethod
    def review_to_sentences( review, tokenizer, remove_stopwords=False ):
        # Function to split a review into parsed sentences. Returns a
        # list of sentences, where each sentence is a list of words
        #
        # 1. Use the NLTK tokenizer to split the paragraph into sentences
        raw_sentences = tokenizer.tokenize(review.strip())
        #
        # 2. Loop over each sentence
        sentences = []
        for raw_sentence in raw_sentences:
            # If a sentence is empty, skip it
            if len(raw_sentence) > 0:
                # Otherwise, call review_to_wordlist to get a list of words
                sentences.append( KaggleWord2VecUtility.review_to_wordlist( raw_sentence, remove_stopwords ))
        #
        # Return the list of sentences (each sentence is a list of words,
        # so this returns a list of lists
        return sentences



In [15]:
clean_review_word = KaggleWord2VecUtility.review_to_wordlist \
        ( train["review"][0], True )
clean_review_sentence = KaggleWord2VecUtility.review_to_wordlist \
        ( train["review"][0], True )


In [16]:
# Get the number of reviews based on the dataframe column size
num_reviews = train["review"].size





In [17]:
print ("Cleaning and parsing the training set movie reviews...\n")
clean_train_reviews = []
for i in range( 0, len(train["review"])):
    if( (i+1)%1000 == 0 ):
        print ("Review %d of %d\n" % ( i+1, num_reviews )  )  
    clean_train_reviews.append(" ".join(KaggleWord2VecUtility.review_to_wordlist(train["review"][i], True)))

Cleaning and parsing the training set movie reviews...

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



In [18]:
clean_train_reviews[0:5]

['stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mj feeling towards press also obvious message drugs bad kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice actual feature film bit finally starts minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond mj overheard plans nah joe pesci character ranted wanted people know supplying drugs etc dunno maybe hates mj music lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually directors hate worki

In [19]:
print ("Creating the bag of words...\n")
from sklearn.feature_extraction.text import CountVectorizer
# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 



Creating the bag of words...



In [20]:
train_data_features = vectorizer.fit_transform(clean_train_reviews)

In [21]:
train_data_features = train_data_features.toarray()

In [22]:
print ("Training the random forest (this may take a while)...")

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100)
  

Training the random forest (this may take a while)...


In [23]:
# Fit the forest to the training set, using the bag of words as
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, train["sentiment"] )

In [24]:
# Create an empty list and append the clean reviews one by one
clean_test_reviews = []

print ("Cleaning and parsing the test set movie reviews...\n")
for i in range(0,len(test["review"])):
    clean_test_reviews.append(" ".join(KaggleWord2VecUtility.review_to_wordlist(test["review"][i], True)))


Cleaning and parsing the test set movie reviews...



In [25]:

# Get a bag of words for the test set, and convert to a numpy array
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()


In [26]:
# Use the random forest to make sentiment label predictions
print ("Predicting test labels...\n")
result = forest.predict(test_data_features)

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

# Use pandas to write the comma-separated output file
output.to_csv('Bag_of_Words_model.csv', index=False, quoting=3)
print ("Wrote results to Bag_of_Words_model.csv")

Predicting test labels...

Wrote results to Bag_of_Words_model.csv


### Word2Vec


In [27]:
#!pip install gensim

In [28]:
import pandas as pd
import os
from nltk.corpus import stopwords
import nltk.data
import logging
import numpy as np  # Make sure that numpy is imported
from gensim.models import Word2Vec
from sklearn.ensemble import RandomForestClassifier

"In lexical analysis, tokenization is the process of breaking a stream of text up into words, phrases, symbols, or other meaningful elements called tokens. The list of tokens becomes input for further processing such as parsing or text mining." -[Wikipedia](https://en.wikipedia.org/wiki/Tokenization_(lexical_analysis)

Punkt is a specific tokenizer.
[http://www.nltk.org/_modules/nltk/tokenize/punkt.html](http://www.nltk.org/_modules/nltk/tokenize/punkt.html) 

In [29]:
# download punkt
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [30]:
#What is a tokenizer 
# http://www.nltk.org/_modules/nltk/tokenize/punkt.html 
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [31]:
# ****** Split the labeled and unlabeled training sets into clean sentences
# Note this will take a while and produce some warnings. 
sentences = []  # Initialize an empty list of sentences

print ("Parsing sentences from training set")
for review in train["review"]:
    sentences += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)


Parsing sentences from training set


/opt/conda/lib/python3.6/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/opt/conda/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


In [32]:
    print ("Parsing sentences from unlabeled set")
    for review in unlabeled_train["review"]:
        sentences += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

Parsing sentences from unlabeled set


/opt/conda/lib/python3.6/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/opt/conda/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/opt/conda/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/opt/conda/lib/python3.6/site-packa

In [33]:
# ****** Define functions to create average word vectors
#

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    #
    # Index2word is a list that contains the names of the words in
    # the model's vocabulary. Convert it to a set, for speed
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set:
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    #
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate
    # the average feature vector for each one and return a 2D numpy array
    #
    # Initialize a counter
    counter = 0.
    #
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    #
    # Loop through the reviews
    for review in reviews:
    # Print a status message every 1000th review
       if counter%1000. == 0.:
            print ("Review %d of %d" % (counter, len(reviews)))
               # Call the function (defined above) that makes average feature vectors
            reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
            # Increment the counter
            counter = counter + 1.
    return reviewFeatureVecs


def getCleanReviews(reviews):
    clean_reviews = []
    for review in reviews["review"]:
        clean_reviews.append( KaggleWord2VecUtility.review_to_wordlist( review, remove_stopwords=True ))
    return clean_reviews

In [34]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality
min_word_count = 40   # Minimum word count
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size
downsampling = 1e-3   # Downsample setting for frequent words

In [35]:
# Initialize and train the model (this will take some time)
print ("Training Word2Vec model...")
model = Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling, seed=1)

2017-12-04 14:16:18,963 : INFO : collecting all words and their counts
2017-12-04 14:16:18,966 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-12-04 14:16:19,035 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2017-12-04 14:16:19,109 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types


Training Word2Vec model...


2017-12-04 14:16:19,185 : INFO : PROGRESS: at sentence #30000, processed 671314 words, keeping 30034 word types
2017-12-04 14:16:19,261 : INFO : PROGRESS: at sentence #40000, processed 897814 words, keeping 34348 word types
2017-12-04 14:16:19,334 : INFO : PROGRESS: at sentence #50000, processed 1116962 words, keeping 37761 word types
2017-12-04 14:16:19,406 : INFO : PROGRESS: at sentence #60000, processed 1338403 words, keeping 40723 word types
2017-12-04 14:16:19,482 : INFO : PROGRESS: at sentence #70000, processed 1561579 words, keeping 43333 word types
2017-12-04 14:16:19,558 : INFO : PROGRESS: at sentence #80000, processed 1780886 words, keeping 45714 word types
2017-12-04 14:16:19,638 : INFO : PROGRESS: at sentence #90000, processed 2004995 words, keeping 48135 word types
2017-12-04 14:16:19,715 : INFO : PROGRESS: at sentence #100000, processed 2226966 words, keeping 50207 word types
2017-12-04 14:16:19,792 : INFO : PROGRESS: at sentence #110000, processed 2446580 words, keeping 

2017-12-04 14:16:24,905 : INFO : PROGRESS: at sentence #750000, processed 16771399 words, keeping 120295 word types
2017-12-04 14:16:24,984 : INFO : PROGRESS: at sentence #760000, processed 16990803 words, keeping 120930 word types
2017-12-04 14:16:25,065 : INFO : PROGRESS: at sentence #770000, processed 17217940 words, keeping 121703 word types
2017-12-04 14:16:25,149 : INFO : PROGRESS: at sentence #780000, processed 17448086 words, keeping 122402 word types
2017-12-04 14:16:25,229 : INFO : PROGRESS: at sentence #790000, processed 17675162 words, keeping 123066 word types
2017-12-04 14:16:25,275 : INFO : collected 123504 word types from a corpus of 17798263 raw words and 795538 sentences
2017-12-04 14:16:25,277 : INFO : Loading a fresh vocabulary
2017-12-04 14:16:25,381 : INFO : min_count=40 retains 16490 unique words (13% of original 123504, drops 107014)
2017-12-04 14:16:25,383 : INFO : min_count=40 leaves 17239118 word corpus (96% of original 17798263, drops 559145)
2017-12-04 14:1

2017-12-04 14:17:32,452 : INFO : PROGRESS: at 57.94% examples, 554401 words/s, in_qsize 7, out_qsize 0
2017-12-04 14:17:33,459 : INFO : PROGRESS: at 58.82% examples, 554396 words/s, in_qsize 8, out_qsize 0
2017-12-04 14:17:34,466 : INFO : PROGRESS: at 59.72% examples, 554708 words/s, in_qsize 7, out_qsize 0
2017-12-04 14:17:35,472 : INFO : PROGRESS: at 60.63% examples, 555010 words/s, in_qsize 8, out_qsize 0
2017-12-04 14:17:36,486 : INFO : PROGRESS: at 61.57% examples, 555441 words/s, in_qsize 7, out_qsize 0
2017-12-04 14:17:37,492 : INFO : PROGRESS: at 62.46% examples, 555526 words/s, in_qsize 7, out_qsize 0
2017-12-04 14:17:38,496 : INFO : PROGRESS: at 63.35% examples, 555544 words/s, in_qsize 8, out_qsize 0
2017-12-04 14:17:39,509 : INFO : PROGRESS: at 64.21% examples, 555369 words/s, in_qsize 7, out_qsize 0
2017-12-04 14:17:40,513 : INFO : PROGRESS: at 65.09% examples, 555377 words/s, in_qsize 7, out_qsize 0
2017-12-04 14:17:41,520 : INFO : PROGRESS: at 66.00% examples, 555639 wor

In [36]:
# If you don't plan to train the model any further, calling
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)



2017-12-04 14:18:20,623 : INFO : precomputing L2-norms of word weight vectors


In [37]:
# It can be helpful to create a meaningful model name and
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

model.doesnt_match("man woman child kitchen".split())



2017-12-04 14:18:20,745 : INFO : saving Word2Vec object under 300features_40minwords_10context, separately None
2017-12-04 14:18:20,748 : INFO : not storing attribute syn0norm
2017-12-04 14:18:20,751 : INFO : not storing attribute cum_table
2017-12-04 14:18:21,426 : INFO : saved 300features_40minwords_10context


'kitchen'

In [38]:
model.doesnt_match("france england germany soccer".split())


'soccer'

In [39]:
model.most_similar("soccer")

[('football', 0.8058714866638184),
 ('basketball', 0.7078469395637512),
 ('baseball', 0.660082221031189),
 ('poker', 0.602813720703125),
 ('hockey', 0.5837126970291138),
 ('wrestling', 0.5702486634254456),
 ('shaolin', 0.5629345774650574),
 ('sports', 0.562635064125061),
 ('coach', 0.5586018562316895),
 ('chess', 0.5561162233352661)]

In [40]:
model.most_similar("man")

[('woman', 0.626766562461853),
 ('lady', 0.615524411201477),
 ('lad', 0.5628026723861694),
 ('businessman', 0.5256783366203308),
 ('monk', 0.523580014705658),
 ('guy', 0.5223524570465088),
 ('men', 0.5216269493103027),
 ('millionaire', 0.5188726186752319),
 ('person', 0.49897968769073486),
 ('boy', 0.4951506555080414)]

In [41]:
model["computer"]

array([ -5.91147952e-02,  -3.44899520e-02,  -2.45185191e-04,
         8.68700370e-02,  -2.48731822e-02,  -8.83342922e-02,
         5.65314256e-02,   5.57818636e-02,  -9.83514171e-03,
        -3.34338434e-02,  -8.98361728e-02,  -3.00597399e-02,
         4.68104780e-02,   8.52110535e-02,   3.27721275e-02,
         6.14302745e-03,  -8.85524154e-02,   9.71228331e-02,
        -4.43680808e-02,   1.26444161e-01,   3.62907723e-02,
         6.06430471e-02,  -1.04446523e-02,  -6.31649718e-02,
         9.27746743e-02,  -1.34493480e-03,  -4.96417396e-02,
        -1.03354372e-01,   5.32339327e-02,   8.46155211e-02,
        -1.01224529e-02,  -7.41985589e-02,  -3.00516635e-02,
         1.12062737e-01,   1.04944119e-02,   5.64037822e-02,
         2.81863976e-02,  -6.49562478e-02,  -5.91108873e-02,
         6.64753690e-02,   4.36288752e-02,  -2.80346833e-02,
         5.17095439e-02,   4.76429984e-02,  -1.72620188e-04,
         3.19770835e-02,   7.58196414e-02,  -1.08765010e-02,
        -6.85802698e-02,

In [42]:
model.most_similar("car")

[('truck', 0.7491682767868042),
 ('bus', 0.6793311834335327),
 ('bike', 0.6504430770874023),
 ('plane', 0.6291133165359497),
 ('train', 0.622740626335144),
 ('jeep', 0.6214756965637207),
 ('chevy', 0.6135284900665283),
 ('garage', 0.6002269387245178),
 ('cab', 0.5953395366668701),
 ('horse', 0.5897566080093384)]